In [31]:
#### SetUp

import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_validate, GridSearchCV, KFold
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.svm import SVC, OneClassSVM
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb

import pywt
import numpy as np
import pandas as pd
import pywt
import seaborn as sns
import scaleogram as scg 
import matplotlib.pyplot as plt
import matplotlib.gridspec as GridSpec
from mat4py import loadmat
from scipy.fftpack import fft

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import matplotlib.pyplot as plt


from collections import Counter
from scipy import stats
from itertools import repeat

import biosppy.signals.ecg as ecg

vColors = ["#049DD9", "#03A64A", "#F2AC29", "#F2CA80", "#F22929"]

from scipy.signal import spectrogram
from sklearn.preprocessing import MinMaxScaler
import neurokit2 as nk
from neurokit2 import ecg_quality, ecg_clean, ecg_process, ecg_analyze, ecg_plot, entropy_sample, entropy_approximate, signal_detrend, signal_power, signal_psd
from joblib import Parallel, delayed
from tqdm import tqdm
import heartpy as hp
from scipy.signal import resample
import math
import biosppy

In [78]:
X_train = pd.read_csv('X_train.csv').drop(labels='id', axis=1).drop("x17978", axis = 1).to_numpy()
y_train = np.ravel(pd.read_csv('y_train.csv').drop(labels='id', axis=1).to_numpy())

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning:

Columns (17979) have mixed types. Specify dtype option on import or set low_memory=False.



In [ ]:
# Filtering comparision

n = 15

row = X_train[n,:][~np.isnan(X_train[n,100:])]

X_train_filtered = biosppy.ecg.ecg(row, sampling_rate=300, show = False)["filtered"]

fig, axs = plt.subplots(2)
axs[0].plot(row, 'tab:orange')
axs[0].set_title("raw")
axs[1].plot(X_train_filtered, 'tab:green')
axs[1].set_title("filtered")

In [ ]:
# Feature Extraction loop

X_train_ecg = []
Fails = list()
    q1
for i in range(0,X_train.shape[0],1):
    if i % 50 == 0:
        print(i)
    
    try:
        X_train_filtered = biosppy.ecg.ecg(X_train[i,:][~np.isnan(X_train[i,:])], sampling_rate=300, show = False)["filtered"]

        resampled_data = resample(X_train_filtered, len(X_train_filtered)*5)
        scaled_data = hp.scale_data(resampled_data)

        try:
            processed_data, info = nk.ecg_process(scaled_data, sampling_rate=300)
        except:
            try:
                processed_data, info = nk.ecg_process(-1*scaled_data, sampling_rate=300)
            except:
                processed_data, info = nk.ecg_process(np.square(scaled_data), sampling_rate=300)

        X_train_ecg.append(nk.bio_analyze(processed_data, sampling_rate=300).iloc[0,:].to_list())
    except:
        Fails.append(i)
        print(i,"did not work")


X_train_ecg = pd.DataFrame(X_train_ecg, columns = list(nk.bio_analyze(processed_data, sampling_rate=300).columns))

[314,
 835,
 976,
 1097,
 1640,
 2106,
 2888,
 3348,
 3933,
 4165,
 4239,
 4970,
 4980,
 5072] in Train Data can not be processed!

83, 1908, 2630, 2828, 2902, 3003, 3357 in Test Data can not be processed!

In [22]:
# For export
#X_train_ecg.to_csv('X_train_ecg.csv')

In [79]:
# Import (again) and deleting not processed rows in response vector

X_train_ecg = pd.read_csv("X_train_ecg.csv").drop(["HRV_ULF","HRV_VLF"], axis=1)
y_train = np.delete(y_train, [314, 835, 976, 1097, 1640, 2106, 2888, 3348, 3933, 4165, 4239, 4970, 4980, 5072], axis=0).ravel()

In [80]:
X_train_ecg = X_train_ecg.replace([np.inf, -np.inf], np.nan)

X_train_ecg.fillna(X_train_ecg.median(), inplace=True)

In [85]:
scaler = StandardScaler() 

X_train_ecg = pd.DataFrame(
    scaler.fit_transform(
    X_train_ecg),
    columns=X_train_ecg.columns)

In [91]:
# If wanted: Feature reduction by lasso (No must)

lsvc = LogisticRegression(C=0.01, penalty='l1', solver = "liblinear").fit(X_train_ecg, y_train)
model = SelectFromModel(lsvc, prefit=True)

X_train_ecg = pd.DataFrame(model.transform(X_train_ecg))

X_train_ecg.shape

(5103, 22)

In [ ]:
F1 = []
cv = KFold(n_splits=5)

model = RandomForestClassifier(n_estimators = 500)

# model = lgb.LGBMClassifier(objective='multiclass',
#                         n_estimators = 50,
#                         learning_rate = 0.1,
#                         random_state=42,
#                         max_depth = 100,
#                         min_child_samples = 2,
#                         min_child_weight = 0,
#                         subsample = 0.8,
#                         colsample_bytree = 0.8,
#                         reg_alpha = 0
#                         )

for train_ix, test_ix in cv.split(X_train_ecg):

    X_cvtrain, X_cvtest = X_train_ecg.iloc[train_ix, :], X_train_ecg.iloc[test_ix, :]
    y_cvtrain, y_cvtest = y_train[train_ix], y_train[test_ix]

    model.fit(X_cvtrain, y_cvtrain)

    predtrain = model.predict(X_cvtrain)
    pred = model.predict(X_cvtest)

    print("\nTrain F1:")
    print(np.round(f1_score(y_cvtrain, predtrain, average='micro'), 4))
    print("\nTest F1:")
    print(np.round(f1_score(y_cvtest, pred, average='micro'), 4))
    print("\n________________________")

    F1.append(np.round(f1_score(y_cvtest, pred, average= "micro"), 4))

print("\nAverage F1:", round(np.sum(F1)/5, 4))
print("Std:", round(np.std(F1), 4))